In [150]:
import mlflow
import os, sys

import torch
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import Image
from mlflow.tracking import MlflowClient

Set MLflow tracking URI:

In [45]:
TRACKING_URI = "file:///home/rodrigo/CISTIB/repos/CardiacMotionRL/mlruns"
mlflow.set_tracking_uri(TRACKING_URI)

In [122]:
experiment_w = widgets.Select(
    options=[exp.name for exp in mlflow.list_experiments()]
)
display(experiment_w)

A Jupyter Widget

In [47]:
exp_id = mlflow.get_experiment_by_name(experiment_w.value).experiment_id

# Retrieving runs

In [104]:
# runs_list = mlflow.search_runs(experiment_ids=[exp_id], output_format="list")
runs_df = mlflow.search_runs(experiment_ids=[exp_id],)
runs_df = runs_df[runs_df.status == "FINISHED"].reset_index(drop=True)

In [105]:
test_ratio_cols = runs_df.columns[runs_df.columns.str.contains("test.*ratio")]
runs_df[test_ratio_cols]

,metrics.test_rec_ratio_to_pop_mean,metrics.test_rec_ratio_to_time_mean,metrics.test_rec_ratio_to_pop_mean_c
0,1.953890,14.61279,0.122813
1,128416.752789,657839.50000,3367.385742


### Getting artifacts

In [51]:
client = MlflowClient()
local_dir = "/tmp/artifact_downloads"

if not os.path.exists(local_dir):
    os.mkdir(local_dir)

In [111]:
client._tracking_client.list_artifacts(
    runs_df.run_id[1]
)

[<FileInfo: file_size=None, is_dir=True, path='animations'>,
 <FileInfo: file_size=None, is_dir=True, path='images'>,
 <FileInfo: file_size=None, is_dir=True, path='model'>,
 <FileInfo: file_size=6899, is_dir=False, path='model_summary.txt'>,
 <FileInfo: file_size=None, is_dir=True, path='restored_model_checkpoint'>]

In [148]:
def display_gif(i):
    kk = client.download_artifacts(
      runs_df.run_id[i], 
      "animations", 
      local_dir
    )
    gif_file = os.path.join(kk, os.listdir(kk)[0])
    gif = Image(data=open(gif_file,'rb').read(), format='png')
    display(gif)    
    

In [149]:
interact(display_gif, i=widgets.IntSlider(min=0, max=1));

A Jupyter Widget

In [126]:
kk = client.download_artifacts(
      runs_df.run_id[0], 
      "model", 
      local_dir
    )

In [128]:
os.listdir(kk)

['data', 'conda.yaml', 'MLmodel', 'requirements.txt']

In [129]:
model = os.path.join(kk, "data/model.pth")

In [151]:
torch.load(os.path.basename(model))

ModuleNotFoundError: No module named 'models'